In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("LinearRegression") \
.master("local[4]") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# Reading Data Set

In [6]:
df = spark.read.format("csv") \
.option("header",True) \
.option("sep",",") \
.option("inferSchema",True) \
.load("Q:\\SparkDatasets\\Advertising.csv")

In [7]:
df.toPandas().head()

,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Changing Column Name

<span style="color:green">Hedef değişkenin ismini "Sales" 'ı "label" olarak değiştiriyorum. </span>

In [8]:
yeni_nitelikler = ["id", "TV","Radio","Newspaper","label"]

In [9]:
df2 = df.selectExpr("_c0 as id", "TV","Radio","Newspaper","Sales as label")

In [10]:
df2.toPandas().head()

,id,TV,Radio,Newspaper,label
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


<span style="color:orange">Nümerik nitelikler ile hedef niteliği ayrı ayrı listelerde tutatlı daha sonra işimize yaracak bunlar. </span>

In [11]:
numerik_nitelikler = ["TV","Radio"]
label = ["label"]

In [12]:
df2.describe().toPandas().head()

,summary,id,TV,Radio,Newspaper,label
0,count,200,200,200,200,200
1,mean,100.5,147.0425,23.264000000000024,30.553999999999995,14.022500000000003
2,stddev,57.879184513951124,85.85423631490805,14.846809176168728,21.77862083852283,5.217456565710477
3,min,1,0.7,0.0,0.3,1.6
4,max,200,296.4,49.6,114.0,27.0


<span style="color:green">Elimizde kategorik değişken olmadıgından stringIndexer ve OneHotEncoding uygulamayacağız. </span>

# Data Preparing

In [13]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler() \
.setInputCols(numerik_nitelikler) \
.setOutputCol("features")

<span style="color:green"> features ' u makine öğrenme modeline girecek en son nitelik ismi olarak veriyoruz biri "features" idi diğeri "label" idi. </span>

# Regression Model

In [14]:
from pyspark.ml.regression import LinearRegression
lr_obj = LinearRegression() \
.setFeaturesCol("features") \
.setLabelCol("label")

# Pipeline Model

In [15]:
from pyspark.ml import Pipeline
pipeline_obj = Pipeline() \
.setStages([vector_assembler, lr_obj])

# Sperate Dataset

In [16]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed=142)

# Train Model

In [17]:
pipeline_model = pipeline_obj.fit(train_df)

<span style="color:red">Modeli pipline içerisinde kullandığımıza dikkat edelim. </span>

# Model Testing

In [28]:
result_df = pipeline_model.transform(test_df)
result=result_df.toPandas()
result.head()

,id,TV,Radio,Newspaper,label,features,prediction
0,3,17.2,45.9,69.3,9.3,"[17.2, 45.9]",12.773880
1,6,8.7,48.9,75.0,7.2,"[8.7, 48.9]",12.991416
2,9,8.6,2.1,1.0,4.8,"[8.6, 2.1]",3.731130
3,10,199.8,2.6,21.2,10.6,"[199.8, 2.6]",12.283048
4,17,67.8,36.6,114.0,12.5,"[67.8, 36.6]",13.171620


In [29]:
pipeline_model.stages

[VectorAssembler_b77800798516, LinearRegression_62efe9dd99e2]

<span style="color:red"> pipeline_model.stages içerisinde <span style="color:blue">VectorAssembler </span> ve <span style="color:blue"> LinearRegression</span> bulunmakta.</span>

<span style="color:red"> Bizim merak ettiğimiz değerler  <span style="color:blue">r2 </span> değeri, <span style="color:blue"> rmsa</span> değeri bunu da <span style="color:blue">LinearRegression </span> içerisinden almalıyız alalım.</span>

<span style="color:green">pipeline_model.stages[1] diyeceğiz çünkü [VectorAssembler_c2f5e5bc864b, LinearRegression_acd90e6acfb2] den görüldüğü gibi LinearRegression  1.indis (index) numaralı elemanıdır. </span>

# Taking Linear Model Inside Pipeline Model (lineer modeli pipeline model içinden almak)

In [30]:
lr_model = pipeline_model.stages[1]

In [31]:
lr_model.coefficients

DenseVector([0.0442, 0.1978])

In [32]:
lr_model.intercept

2.935593134859488

In [33]:
lr_model.summary.r2

0.8928931248714045

<span style="color:red">Toplam değişkenliğin %89 unu açıklayabiliyor modelimiz. </span>

<span style="color:blue">En az bir p değeri anlamlı ise model anlamlı sayılabilir. datalegs </span>

In [34]:
lr_model.summary.rootMeanSquaredError

1.6578475603790448

In [35]:
lr_model.summary.pValues

[0.0, 0.0, 3.774758283725532e-15]

In [36]:
lr_model.summary.tValues

[27.918094216203865, 21.216582516976807, 8.740412243937218]

In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# word cloud library
#from wordcloud import WordCloud

# matplotlib
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [39]:
# create trace 1 that is 3d scatter
trace1 = go.Scatter3d(
    x=result.prediction,
    y=result.label,
    z=result.Newspaper,
    mode='markers',
    marker=dict(
        size=10,
        color=result.Radio,                # set color to an array/list of desired values      
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    )
    
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

 # Choosing Model

* Şimdi geriye dogru eleme yöntemi ile model seçim yöntemi kullanacağız.Bu yöntemde modele tüm değişkenleri dahil ediyorduk daha sonra pValues 'lara bakıyorduk pValues 'dan eşik değerin üzerinde kalan değerin üzerinde kalan değeri en büyük değeri çıkararak birer birer modeli eğitiyorduk.

* Genelde PValues 0.05 kabul edilir anlamlı olması için bundan küçük olmalı

* lr_model.summary.pValues daki bulunan ,

* [0.0, 0.0, 0.5717102604020492, 3.774758283725532e-15]

* değerler olacaktı numerik_nitelikler = ["TV","Radio","Newspaper"] olsaydı.

* "Newspaper" ı yukarıda ,

* numerik_nitelikler = ["TV","Radio"]
 
* şeklinde bir düzenleme yaptık çünkü pValues değeri 0.05 den büyük oldugundan çıkardık. 
 
* birinci pValues "0.0" birinci niteliğe ait "TV" ,
* ikinci pValues "0.0" ikinci niteliğe ait "Radio" ,
* üçüncü pValues "0.5717102604020492" üçüncü niteliğe ait "Newspaper" olacaktı fakat çıkardık bu değeri,
* dördüncü pValues "3.774758283725532e-15" dördüncü niteliğe ait "sabit e ait",
* Kısaca buraya kadar numerik nitelikleri ikiye düşürmüş olduk.


<span style="color:blue">Geriye doğru eleme yönteminde Newspaper p değeri 0.05 değerinden büyük olduğu (0.5717) için modelden çıkarılır. Bunun için nümerik nitelikler listesinden Newspaper'ı çıkarmamız yeterli. Daha sonra o paragraftan itibaren tekrar çalıştıralım.</span>

İkinci denemede tüm p değerleri eşik değer altında kaldığı için regresyon modeli: 

<span style="color:red">y = 2.935593134859488 + (0.0442 * TV) + (0.1978 * Radio)  </span>

# Prediction

In [40]:
import pandas as pd
d = {'TV': [100.0,250.0,150.0,200.0,300.0,345.0,420.0], 'Radio': [10.0,25.0,400.0,600.0,25.0,15.0,5.0]}
pd_df = pd.DataFrame(data=d)

In [41]:
pd_df.head()

,TV,Radio
0,100.0,10.0
1,250.0,25.0
2,150.0,400.0
3,200.0,600.0
4,300.0,25.0


In [42]:
predict_df = spark.createDataFrame(pd_df)

In [43]:
predict_df.show()

+-----+-----+
|   TV|Radio|
+-----+-----+
|100.0| 10.0|
|250.0| 25.0|
|150.0|400.0|
|200.0|600.0|
|300.0| 25.0|
|345.0| 15.0|
|420.0|  5.0|
+-----+-----+



In [51]:
predict_vec_df = vector_assembler.transform(predict_df)

In [52]:
res=lr_model.transform(predict_vec_df)

In [53]:
res.show()

+-----+-----+-------------+------------------+
|   TV|Radio|     features|        prediction|
+-----+-----+-------------+------------------+
|100.0| 10.0| [100.0,10.0]| 9.334383277881834|
|250.0| 25.0| [250.0,25.0]|18.932568492415353|
|150.0|400.0|[150.0,400.0]| 88.67711459534111|
|200.0|600.0|[200.0,600.0]|130.44261503817665|
|300.0| 25.0| [300.0,25.0]|  21.1430890672259|
|345.0| 15.0| [345.0,15.0]|21.154808591154143|
|420.0|  5.0|  [420.0,5.0]| 22.49284045996872|
+-----+-----+-------------+------------------+



In [54]:
pandas_dataframe = res.select("*").toPandas()

In [60]:
# Creating trace1
trace_m = go.Scatter(
                    x = pandas_dataframe.prediction,
                    y = pandas_dataframe.TV,
                    mode = "lines",
                    name = "TV",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= "TV")

In [61]:
 # Creating trace2
trace_n = go.Scatter(
                    x = pandas_dataframe.prediction,
                    y = pandas_dataframe.Radio,
                    mode = "lines+markers",
                    name = "Radio",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "RADIO")

In [64]:
data = [trace_m, trace_n]
layout = dict(title = ' TV and Radio Advertisement Effect on Sales',
              xaxis= dict(title= 'Prediction',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)